In [2]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("de") # load a new spacy model
db = DocBin() # create a DocBin object

loading full_matcher ...
loading abv_matcher ...
loading full_uni_matcher ...
loading low_form_matcher ...
loading token_matcher ...


In [6]:
import json
f = open('training_data.json')
TRAIN_DATA = json.load(f)


In [7]:
for text, annot in tqdm(TRAIN_DATA['annotations']): 
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            pass
        else:
            ents.append(span)
    doc.ents = ents 
    db.add(doc)

db.to_disk("./training_data.spacy") # save the docbin object

100%|██████████| 53/53 [00:00<00:00, 1263.22it/s]


In [8]:
! python -m spacy init config config.cfg --lang de --pipeline ner --optimize efficiency

⚠ To generate a more effective transformer-based config (GPU-only), install the
spacy-transformers package and re-run this command. The config generated now
does not use transformers.
ℹ Generated config template specific for your use case
- Language: de
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [9]:
! python -m spacy train config.cfg --output ./custom_model --paths.train ./training_data.spacy --paths.dev ./training_data.spacy


✔ Created output directory: custom_model
ℹ Saving to output directory: custom_model
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     49.76    0.00    0.00    0.00    0.00

[2022-06-11 20:25:17,118] [INFO] Set up nlp object from config
[2022-06-11 20:25:17,126] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-06-11 20:25:17,130] [INFO] Created vocabulary
[2022-06-11 20:25:17,131] [INFO] Finished initializing nlp object
[2022-06-11 20:25:17,267] [INFO] Initialized pipeline components: ['tok2vec', 'ner']



 17     200        896.35   1861.46   98.31   98.31   98.31    0.98
 38     400         16.18     32.80  100.00  100.00  100.00    1.00
 65     600          3.50      2.37  100.00  100.00  100.00    1.00
 98     800          0.00      0.00  100.00  100.00  100.00    1.00
138    1000          0.00      0.00  100.00  100.00  100.00    1.00
187    1200          0.00      0.00  100.00  100.00  100.00    1.00
249    1400          0.00      0.00  100.00  100.00  100.00    1.00
315    1600          0.00      0.00  100.00  100.00  100.00    1.00
411    1800        130.93     61.66  100.00  100.00  100.00    1.00
511    2000        174.48     45.88  100.00  100.00  100.00    1.00
✔ Saved pipeline to output directory
custom_model\model-last


In [10]:
nlp_ner = spacy.load("C:/Users/Aleksej Aikov/Desktop/Enablement/Master/04_Analysis/04_03_extracting_skills_with_skillner_ner/04_03_01_custom_ner_example/custom_model/model-best")


In [11]:
doc = nlp_ner("""Erfolgreich abgeschlossenes Studium mit der Fachrichtung Mathematik, Statistik, IT, Finanzen oder einem ähnlichen Bereich • Idealerweise beherrschst du die Tableau Software, MS PowerPoint Plugins und MS Power Automate • Kenntnisse mit VBA/Power Query/REST API • Erfahrung mit Projektmanagement-Tools wie ServiceNow und Jira • Starke analytische und numerische Fähigkeiten, um komplexe Fragen in einfache Lösungen umwandeln zu können • Gute Kommunikationsfähigkeit, service- und lösungsorientierte Arbeitshaltung sowie hohe analytische Fähigkeiten • Fließende Englischkenntnisse • Reisebereitschaft Vollständige Stellenbeschreibung Das sind deine Aufgaben • Ausbau und Weiterentwicklung der Analyseinstrumente und -methoden mit Ausrichtung auf die Anforderungen der Fachbereiche und Business Partner • Analyse und Verknüpfung verschiedener Datenquellen und Bearbeitung großer Datenmengen • Verwendung geeigneter Algorithmen zur Lösung von konkreten Fragestellungen in Proof-of-Concepts und Validierung mit Fachexperten • Entwicklung und Optimierung von Data Analytics und ML Modellen in Piloten bis zur Produktion • Dokumentation und Pflege von Experimenten, Modellen, KPIs in modernen Frameworks • Unterstützung der internen Kollegen bei der Industrialisierung von Modellen sowie bei der Interpretation und Visualisierung von Ergebnissen • Mitarbeit in cross-funktionalen Projektteams zur Realisierung von Use Cases Das bringst du mit • Erfolgreich abgeschlossenes Studium der Fachrichtungen Mathematik, Informatik, Physik, Wirtschaftsinformatik oder eine vergleichbare Qualifikation • Mehrjährige Berufserfahrung in der Analyse komplexer Datensätze und in Anwendung von Data Science, Machine Learning, Data Mining, Optimierungs-Methoden • Experte im Umgang mit gängigen Plattformen und Tools (z.B. Azure Databricks, Jupyter, KNIME) und Sprachen (z.B. Python, R, SQL) • Erfahrung in Visualisierung von Daten und Ergebnissen in Python/R sowie PowerPoint, Excel oder Power BI • Erste Erfahrung im LEH/ Einzelhandel, idealerweise im intern. Kontext • Sehr gute Deutsch- und Englischkenntnisse in Wort und Schrift Vollständige Stellenbeschreibung """)

In [12]:
spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter


In [19]:
doc2 = nlp_ner("""Sie haben ein abgeschlossenes Studium als (Wirtschafts-) Informatiker:in oder eine gleichwertige Qualifikation
Sie bringen bereits Berufserfahrung in den relevanten Aufgabengebieten mit
Erfahrungen im Projektmanagement und Prozessaffinität sind vorhanden
Fundierte Kenntnisse in MS-Office, insb. Excel (VBA), Datenbankmanagementsystemen / SQL sowie MS Power BI
Teamfähigkeit, gute organisatorische und kommunikative Fähigkeiten runden Ihr Profil ab
Vertrauen Sie auf die Sicherheit und Zuverlässigkeit eines großen kommunalen Arbeitgebers, eine attraktive und faire Vergütung und den Komfort einer betrieblichen Altersversorgung.
Ihre Aufgabe ist für die Zukunft unserer Region von Bedeutung. Dafür ziehen wir alle an einem Strang und nutzen die Stärken des gesamten Teams.
Rechnen Sie mit kollegialer Unterstützung, Duz-Kultur und Menschen, die den konstruktiven Austausch mit Ihnen suchen.
Passen Sie Ihren Beruf Ihrem Leben an, denn mit unseren flexiblen Arbeitszeitmodellen bestimmen Sie den Rhythmus Ihres beruflichen Alltags selbst. Auch Ihr Nachwuchs liegt uns am Herzen: Bei unseren „Stadtwerke Minis“ werden sich Ihre Liebsten garantiert wohlfühlen.
Entwickeln Sie sich bei uns: Kontinuierlich und regelmäßig nehmen wir die persönliche und fachliche Entwicklung in den Blick. So erreichen Sie mit uns Ihre Ziele.
Nutzen Sie unser umfangreiches Paket zur Gesundheitsförderung: Dabei sind Firmenfitness (Hansefit/ Qualitrain), diverse Präventionsprogramme und Beratungsangebote nur Auszüge, denn Gesundheit steckt bei uns in jedem Arbeitsplatz.
Genießen Sie den vergünstigten Bezug von Strom und Wasser oder die Möglichkeiten zur mobilen Arbeit
Du besitzt ein ausgezeichnet abgeschlossenes Studium in Data Science, (Wirtschafts-)Informatik, (Wirtschafts-)Mathematik, Statistik, Ökonometrie – alternativ überzeugst du mit einem vergleichbaren Studium
Deine sehr guten Kenntnisse in der Datenaufbereitung, dem Aufsetzen von ETL-Prozessen, der Datenanalyse und -visualisierung mit Python bzw. PySpark und SQL sind bei uns gefragt
Du kennst dich mit cloudbasierter Infrastruktur und Cloud Computing aus, idealerweise mit Microsoft Azure - Erste Erfahrungen mit Cloud-Services wie Databricks oder der Azure Data Factory hast du idealerweise bereits gesammelt
Dich zeichnet deine Leidenschaft für Data Science aus und du bringst eine hohe Motivation mit, um gemeinsam im Team die datengetriebenen Prozesse voranzubringen
Sehr gute verhandlungssichere Deutschkenntnisse sowie gute Englischkenntnisse runden neben deiner Lernbereitschaft dein Profil ab 
Du verfügst über ein abgeschlossenes informationstechnisches Studium oder eine vergleichbare Ausbildung? Du hast bereits tiefgreifende Fähigkeiten im Bereich Data Warehousing und Analyse? Du interessierst Dich für das Online Business, BI- und Big Data-Technologie? Du siehst Dich in den unten genannten Punkten beschrieben? Dann freuen wir uns auf Deine Bewerbung.
Du zeichnest Dich durch mehrjährige Berufserfahrung im Bereich Data Warehousing, Reporting und Analytics aus.
Du beherrschst die SQL Server Plattform, insbesondere SSIS und kennst Dich mit Tableau als Reporting Tools aus.
Die Nutzung von SQL gehört für Dich zum Arbeitsalltag.""")

In [20]:
spacy.displacy.render(doc2, style="ent", jupyter=True) # display in Jupyter

In [25]:
# imports
import spacy
from spacy.matcher import PhraseMatcher

# load default skills data base
from skillNer.general_params import SKILL_DB
# import skill extractor
from skillNer.skill_extractor_class import SkillExtractor

# init params of skill extractor
nlp = spacy.load("en_core_web_lg")
# init skill extractor
skill_extractor = SkillExtractor(nlp, SKILL_DB, PhraseMatcher)


loading full_matcher ...
loading abv_matcher ...
loading full_uni_matcher ...
loading low_form_matcher ...
loading token_matcher ...


In [ ]:
job_description = """
Successfully completed degree in mathematics, statistics, IT, finance or a similar field
Ideally proficiency in using the Tableau software, MS PowerPoint plugins and MS Power Automate
Knowledge of VBA/Power Query/REST API
Experience in project management tools such as ServiceNow and Jira
Strong analytical and numerical skills that help to translate complex questions into simple solutions
Good communication skills, service and solution-oriented working attitude and remarkable analytical skills
Fluency in English
Willingness to travel
"""

annotations = skill_extractor.annotate(job_description)

In [26]:
skill_extractor.describe(annotations)